In [ ]:
import sys
import sklearn
import os
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from functools import partial
import tensorflow as tf
from tensorflow import keras
import seaborn as sns
import pandas as pd
from tensorflow.keras.utils import to_categorical
from random import shuffle
import glob
from keras.preprocessing import image
from tqdm import tqdm

from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, multilabel_confusion_matrix

In [10]:
# 1. Parameters
batch_size  = 32
img_height  = 150
img_width   = 150
train_dir   = "../DATA/Fruits_Dataset_Train/"
test_dir    = "../DATA/Fruits_Dataset_Test/"
train_csv   = "../DATA/labels_Train.csv"
test_csv    = "../DATA/labels_Test.csv"

# 2. Load CSVs
train_df = pd.read_csv(train_csv)
test_df  = pd.read_csv(test_csv)

# Extract the list of fruit columns (everything except FileName)
fruit_cols = train_df.columns.drop("FileName")

# 3. Build a filename→full‑path map for train & test
all_train_files = glob.glob(os.path.join(train_dir, '*', '*.jpg'))
all_test_files  = glob.glob(os.path.join(test_dir,  '*', '*.jpg'))

train_map = {os.path.basename(p): p for p in all_train_files}
test_map = {os.path.basename(p): p for p in all_test_files}

train_df['filepath'] = train_df['FileName'].map(train_map)
test_df['filepath'] = test_df['FileName'].map(test_map)

# (Optional sanity check: make sure none went missing)
missing = train_df['filepath'].isna().sum()
if missing:
    raise ValueError(f"{missing} train images not found on disk!")

y_train = train_df[fruit_cols].values.astype("float32")  # shape (N_train, 18)
y_test = test_df [fruit_cols].values.astype("float32")  # shape (N_test, 18)

# 4. Create tf.data.Dataset
def make_dataset(filepaths, labels, shuffle=True):
    ds = tf.data.Dataset.from_tensor_slices((filepaths, labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(filepaths))
    def _load_and_preprocess(path, label):
        # Read & decode
        img = tf.io.read_file(path)
        img = tf.image.decode_jpeg(img, channels=3)
        # Resize & scale
        img = tf.image.resize(img, [img_height, img_width])
        img = preprocess_input(img)           # ResNet50 preprocessing
        return img, label
    ds = ds.map(_load_and_preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return ds

train_ds = make_dataset(train_df["filepath"].values, y_train, shuffle=True)
val_ds   = make_dataset(test_df ["filepath"].values, y_test,  shuffle=False)

In [11]:
# 5. Build the model
base_model = ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=(img_height, img_width, 3)
)
base_model.trainable = False  # freeze

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
# For multi‑label: use sigmoid
output = Dense(len(fruit_cols), activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=output)
model.compile(
    optimizer=Adam(1e-4),
    loss='binary_crossentropy',  # per‑label BCE
    metrics=[
        tf.keras.metrics.BinaryAccuracy(name='bin_acc'),
        tf.keras.metrics.Precision(name='prec'),
        tf.keras.metrics.Recall(name='rec'),
        tf.keras.metrics.AUC(name='auc')
    ]
)
model.summary()

# 6. Initial training
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_auc', mode='max',
    patience=3, restore_best_weights=True
)
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=15,
    callbacks=[early_stop]
)

# 7. Fine‑tuning
for layer in base_model.layers[-30:]:
    layer.trainable = True

model.compile(
    optimizer=Adam(1e-5),
    loss='binary_crossentropy',
    metrics=['bin_acc','prec','rec','auc']
)
fine_tune_history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=12,
    callbacks=[early_stop]
)

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 150, 150,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 156, 156,  │          0 │ input_layer_1[0]… │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 75, 75,    │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 75, 75,    │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 75, 75,    │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 77, 77,    │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 38, 38,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 38, 38,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 38, 38,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 38, 38,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 38, 38,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 38, 38,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 38, 38,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 38, 38,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 38, 38,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 38, 38,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 38, 38,    │      1,024 │ conv2_block1_3_c

 Total params: 25,706,388 (98.06 MB)

 Trainable params: 2,118,676 (8.08 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

Epoch 1/15
 37/529 ━━━━━━━━━━━━━━━━━━━━ 9:47 1s/step - auc: 0.6569 - bin_acc: 0.7201 - loss: 0.5681 - prec: 0.4274 - rec: 0.2471

KeyboardInterrupt: 

In [ ]:
# 8. Plot learning curves
import matplotlib.pyplot as plt

def plot_learning_curves(hist, title_suffix=""):
    fig, axes = plt.subplots(1,2,figsize=(14,5))
    # Loss
    axes[0].plot(hist.history['loss'], label='train_loss')
    axes[0].plot(hist.history['val_loss'], label='val_loss')
    axes[0].set_title('Loss'+title_suffix)
    axes[0].legend()
    # AUC
    axes[1].plot(hist.history['auc'], label='train_auc')
    axes[1].plot(hist.history['val_auc'], label='val_auc')
    axes[1].set_title('AUC'+title_suffix)
    axes[1].legend()
    plt.show()

plot_learning_curves(history,    " (Initial)")
plot_learning_curves(fine_tune_history, " (Fine‑tune)")

# 9. Evaluation: threshold at 0.5
y_pred_probs = model.predict(val_ds)
y_pred = (y_pred_probs > 0.5).astype(int)

# True labels
y_true = np.vstack([y for x,y in val_ds])

# 10. Classification report
print(classification_report(
    y_true, y_pred,
    target_names=fruit_cols,
    zero_division=0
))

# 11. Per‑class confusion matrices
cms = multilabel_confusion_matrix(y_true, y_pred)
for idx, fruit in enumerate(fruit_cols):
    tn, fp, fn, tp = cms[idx].ravel()
    print(f"{fruit}: TP={tp}, FP={fp}, TN={tn}, FN={fn}")